## Notes

GetBCoeffs

#### Project Progress :

1. ~~find zero-crossings~~
2. ~~find cycles~~
3. ~~choose representative cycles based on biases~~
4. form a model with bcoeffs for each representative cycle
5. interpolate between each cycle using bcoeffs for amplitude shape and glissando for pitch
6. test accuracy to original sound
7. Tokenize and train model
8. Profit?
   
#### Ideas

Test with no interpolation between the representative cycles or by sample-and-hold the cycles for a duration

What if instead of tokenizing the b-spline coefficients we use the overtones or harmonics of each cycle? At the end of the day, we know that a neural network can distinguish pitch visually.

What if we tokenize the information of the differencs in the b-splines

To better find cycles: group cycles by similarity by using a dot product, larger dot products are much more similar. Find one from the largest group as your repcycle. Make a Union of the groups to find the group whose cycles covers the whole segment.

Compare cycles of next segment with the repcycle of the previous 

In [24]:
# Testing for Pytorch Audio prcessing
# Andrei Cartera 
import os
import numpy as np
#from pathlib import Path
import matplotlib.pyplot as plt
import torchaudio.transforms as T
import torch.nn.functional as nnF
#from IPython.display import Image
import heapq
import math

import torch
import torchaudio 

# Install soundfile and ffmpeg-python if not already installed
# pip install soundfile
# pip install ffmpeg-python

#print(torch.__version__)
print(str(torchaudio.list_audio_backends()))

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)

['soundfile']


In [25]:
AUDIO_PATH = "./audio/"
FILE_NAME = "zero.wav"
AUDIO_SIZE = 16000
SEGMENT_LENGTH = 500
SEGMENT_NUM = 8
NSR_FACTOR = 3.14
CYCLE_EPSILON = 40
FFT_SIZE = 1024

SHOW_WAV = False
SHOW_FFT = False
SHOW_REPC = False

F0_FREQ_RANGE = (50,200)

# how much to conisder frequency, loudness, or middleness, higher values means less adherence
F_SCORE_WEIGHT = 1 # Frequency 
R_SCORE_WEIGHT = 1 # RMS loudness
M_SCORE_WEIGHT = 1 # Middle

In [26]:
# DO NOT EDIT
LOW_F0_INDEX = math.floor(FFT_SIZE/AUDIO_SIZE * F0_FREQ_RANGE[0])
HIGH_F0_INDEX = math.floor(FFT_SIZE/AUDIO_SIZE * F0_FREQ_RANGE[1])

### Helper Functions and Misc

In [27]:
# gets k largest elements from a list along with its indices 
def klargest_with_indices(arr, k, range=None): 
  heap = []
  for i, val in enumerate(arr):
      if range and not (range[0] <= i <= range[1]):
        continue
      heap.append((i,val))
  return heapq.nlargest(k, heap, key=lambda x: x[1])

def calculate_rms(audio):
  return np.sqrt(np.mean(audio**2))

def find_factor_pairs(n):
  factor_pairs = []
  for i in range(1, int(n**0.5) + 1):
    if n % i == 0:
      factor_pairs.append((i, int(n // i)))
  return factor_pairs

# returns an array of RMS values from a signal over time specified by the window size 
def rms_over_windows(waveform_arr, window_size=100, silence_threshold = None):
  rms_values = []
  excluded_ranges = []
  
  start = 0
  while start < len(waveform_arr):
    end = start + window_size
    if end > len(waveform_arr):
      end = len(waveform_arr)
    window = waveform_arr[start:end]
    rms_value = calculate_rms(window)
    rms_values.append(rms_value)
    
    if silence_threshold:
      if rms_value < silence_threshold:
        excluded_ranges.append((start, end))

    start = end
  
  return rms_values, excluded_ranges

def resize_spectrogram(spectrogram, target_size=(224, 224)):
  # Interpolate to the target size
  resized_spectrogram = nnF.interpolate(spectrogram.unsqueeze(0), size=target_size, mode='bilinear', align_corners=False)
  return resized_spectrogram.squeeze(0)

def show_fft(fft_mag, start_sample:int, top:int=0):
  from matplotlib.ticker import ScalarFormatter
  
  # make x-axis frequencies
  frequencies = np.linspace(0, AUDIO_SIZE / 2, FFT_SIZE // 2)
  out_path = f".\\figures\\fft_{FILE_NAME.split('.')[0]}{start_sample}.png"
  # Convert magnitude to dB
  #fft_dB = 20 * np.log10((2/SEGMENT_LENGTH) * fft_mag)

  # Plot the FFT magnitude as a bar plot
  fig = plt.figure(figsize=(40, 10))
  ax = fig.add_subplot(111)
  #plt.plot(frequencies, fft_dB)
  plt.bar(frequencies, fft_mag, width=15, align='center', alpha=0.75)
  plt.xscale('log')
  plt.xticks([50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 3000, 4000, 5000, 8000 ],
             ['50','100','200','300','400','500','600','700','800','900','1000','1500','2000','3000','4000','5000','8000']) 
  plt.xticks(rotation=45)  # Rotate by 45 degrees
  plt.gca().xaxis.set_major_formatter(ScalarFormatter())
  plt.title(f"FFT of \"{FILE_NAME}\" from {start_sample} to {start_sample+SEGMENT_LENGTH} samples")
  plt.xlabel("Frequency (Hz)")
  plt.ylabel("Magnitude")
  if(top):
    largest = klargest_with_indices(fft_mag, top, (LOW_F0_INDEX, HIGH_F0_INDEX))
    for index,value in largest:
      max_x = AUDIO_SIZE/FFT_SIZE * index
      max_y = value
      ax.plot(max_x, max_y, 'ro')
      ax.annotate(f'{max_x} Hz', xy=(max_x, max_y), xytext=(max_x+max_x/15, max_y),color='red')
  plt.grid(True)
  plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
  plt.close()
  # Open the saved image with the default viewer
  if SHOW_WAV:
    if os.path.exists(out_path):
      if os.name == 'nt':  # For Windows
        os.startfile(out_path)

  # Plot the waveform
def show_waveform(waveform_arr, start_sample, zero_crossings:np.array, repcycle=None):

  out_path = f".\\figures\\{FILE_NAME.split('.')[0]}{start_sample}.png"

  fig = plt.figure(figsize=(100, 10))
  ax = fig.add_subplot(111)
  plt.plot(range(start_sample, start_sample+len(waveform_arr)), waveform_arr)
  y_max = max(abs(np.amin(waveform_arr)), abs(np.amax(waveform_arr)))

  if y_max < 0.1: 
    y_max = 0.1
    plt.ylim(-y_max, y_max)
  if np.size(zero_crossings, 0) > 0:
    plt.vlines(zero_crossings, -y_max, y_max, colors='red', linestyles="dashed", linewidth=1)

  if(repcycle):
    #ax.fill_between(t, 1, where=s > 0, facecolor='green', alpha=.5)
    #ax.fill_between(t, -1, where=s < 0, facecolor='red', alpha=.5)
    ax.fill_between(repcycle, -y_max, y_max, facecolor='green', alpha=.25)
  plt.xticks(np.arange(start_sample, start_sample+len(waveform_arr), 100))
  plt.title(f"Waveform of \"{FILE_NAME}\"")
  plt.xlabel("Samples")
  plt.ylabel("Amplitude")
  plt.grid()
  plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
  plt.close()
  # Open the saved image with the default viewer
  if SHOW_FFT:
    if os.path.exists(out_path):
      if os.name == 'nt':  # For Windows
        os.startfile(out_path)

def fft_max(waveform_arr : np.array):

  # Pad with zeros if necessary 
  if(len(waveform_arr) != FFT_SIZE):
    size_diff = abs(len(waveform_arr)-FFT_SIZE)
    list(waveform_arr).extend([0] * size_diff)

    # perform fft and get magnitude
  fft = np.fft.fft(waveform_arr, FFT_SIZE)
  fft_mag = np.abs(fft)
  
    #turn back to np.array
  fft_mag = fft_mag[:FFT_SIZE // 2]
  max_index = np.argmax(fft_mag)
  return fft_mag, max_index
  
def show_repcycles(repcycles, waveform_arr):
  # Find a pair of multiples of n_segments for dysplaing grid of waveforms 
  n_segments = AUDIO_SIZE / SEGMENT_LENGTH
  min_diff = np.inf
  row_col_pair = None
  for pair in find_factor_pairs(n_segments):
    diff = abs(pair[0]-pair[1])
    if(diff < min_diff):
      row_col_pair = pair
      min_diff = diff
  nrows, ncols = pair

  ### make grid plot
  _, axes = plt.subplots(nrows, ncols, figsize=(100, 50))

  axes = axes.flatten()
  # Loop through the data and corresponding axes to plot
  for i, ax in enumerate(axes):
    if i < len(repcycles):

      if repcycles[i]:
        
        curr_repcycle = repcycles[i]
        repc_start = math.floor(curr_repcycle[0])
        repc_end = math.floor(curr_repcycle[1])
        ticks = np.linspace(repc_start, repc_end, 10)
        ax.plot(range(repc_start, repc_end), waveform_arr[repc_start:repc_end])
        ax.set_xticks(ticks)
        y_max = max(abs(np.amin(repcycles[i])), abs(np.amax(repcycles[i])))   

        if y_max < 0.1: 
          y_max = 0.1
          plt.ylim(-y_max, y_max)

        ax.grid(True)

      else:
        ax.text(0.5, 0.5, 'NONE', fontsize=50, ha='center', va='center', alpha=0.5)
    else:
      ax.axis('off') 

  out_path = f".\\figures\\{FILE_NAME}_repc_segmentsize{SEGMENT_LENGTH}.png"

  plt.tight_layout()
  plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
  plt.close()
  # Open the saved image with the default viewer
  if SHOW_REPC:
    if os.path.exists(out_path):
      if os.name == 'nt':  # For Windows
        os.startfile(out_path)

In [28]:
def find_zerocrossings(waveform_arr:np.array, start_sample, dynamic_threshold=0.01):
  last=None 
  zerocrossings = np.array([], dtype='f')
  
  #Filter out quiet parts of the signal using a RMS with windowing
  _, excluded_ranges = rms_over_windows(waveform_arr, 100, dynamic_threshold)

  # turn the excluded ranges into an array of samples that shouldn't be processed
  excluded_samples = [] 
  for start, end in excluded_ranges:
    excluded_samples.extend(range(start_sample + start, start_sample + end))

  #sweep the whole segment
  for index, value in enumerate(waveform_arr):
    absolute_index = start_sample + index
    if absolute_index in excluded_samples:
      continue 
    if last == None:  #record last sample
      last = value
    #elif value == 0:
    #  if np.sign(last) == -1.0 and np.sign(waveform_arr[index+1]) == 1.0:
    #    np.append(zerocrossings, start_sample+index)
    
    # if last sample is negative and current one is positive a zero crossing occured
    if np.sign(last) == -1.0 and np.sign(value) == 1.0: 
      # x = x1 + (x2-x1)/(y2-y1)*(y-y1)
      inter_x = (index-1) + (-last/(value-last))
      zerocrossings = np.append(zerocrossings,start_sample+inter_x)
    last = value
  return zerocrossings

In [29]:
def find_cycles_f0(f0, start_sample, zero_crossings:list):
  
  cycles = []
  f0_length = 1.0/f0 * AUDIO_SIZE #length of samples of fundamental frequency
  last_sample = start_sample*SEGMENT_LENGTH + SEGMENT_LENGTH 
  for start in zero_crossings:
    cyclef0_end = start+f0_length # where we are expecting the cycle to end 
    
    if cyclef0_end > last_sample+CYCLE_EPSILON: #don't look past end of segment, unless CYCLE_EPSILON allows you to see the end 
      continue

    high = cyclef0_end + CYCLE_EPSILON #acceptabale ranges
    low = cyclef0_end - CYCLE_EPSILON
    
    #get values that fall withing the f0+epsilon range
    try:
      possible_cycles = list(filter(lambda x, high=high, low=low: low <= x <= high ,zero_crossings))
      if not possible_cycles:
        #print(f"Failed to find a cycle for sample {start} with an epsilon of {CYCLE_EPSILON}")
        continue
      if len(possible_cycles) > 1:
        # tie-breaker get the end sample that is closest to the expected
        differences = list(map(lambda x, y=cyclef0_end: abs(y-x), possible_cycles))
        end = possible_cycles[np.argmin(differences)]
      else:
        end = possible_cycles[0]
    except Exception as e:
      print(f"Error: Failed to find a cycle for sample {start} with an epsilon of {CYCLE_EPSILON}\n",e)
    cycles.append((start,end))
  return cycles
      

In [30]:
def find_repcycle(segment_wav, start_sample, f0, cycles):
  #score the cycles based on they adherence to the following biases:
  # 1. closeset to f0
  # 2. loudness
  # 3. most in the middle
  mid_segment = start_sample+ (SEGMENT_LENGTH/2);
  f0_length = 1.0/f0 * AUDIO_SIZE #length of samples of fundamental frequency
  
  #get length, RMS, and middle of all cycles 
  fRm_score = []
  for index, x in enumerate(cycles):
    #get length of the cycle and comapre with fundamental frequency reciprocal 
    freq = x[1] - x[0]
    f_score = abs(freq - f0_length)

    #get middle and comare with middle of the whole segment  
    middle = (x[0] + x[1]) / 2
    m_score = abs(middle - mid_segment)

    #get rms
    i_beginning = math.floor(x[0]-start_sample)
    i_end = math.floor(x[1]-start_sample)
    r_score = 1-calculate_rms(segment_wav[i_beginning:i_end])

    fRm_score.append((index, f_score*F_SCORE_WEIGHT, r_score*R_SCORE_WEIGHT, m_score*M_SCORE_WEIGHT))

  sorted_list = sorted(fRm_score, key=lambda x: x[1]+x[2]+x[3])
  return cycles[sorted_list[0][0]]


### Main

In [31]:

waveform, sample_rate = torchaudio.load(AUDIO_PATH + FILE_NAME)
waveform_arr = waveform.t().squeeze(1).numpy()

#identify SNR reciprocal to get a dynamic noise threshold for filtering 
rms_values, _ = rms_over_windows(waveform_arr, SEGMENT_LENGTH) #calculate RMS over time

# Get Noise-to-Signal Ratio
signal_rms = np.max(rms_values)
noise_rms = np.min(rms_values)
nsr = noise_rms/ signal_rms

# Calculate dynamic silence threshold based on NSR
silence_threshold = nsr*NSR_FACTOR

# Split the waveform into segments
split_waveform = np.array_split(waveform_arr, np.arange(SEGMENT_LENGTH, AUDIO_SIZE, SEGMENT_LENGTH))
#split_waveform = [waveform_np[i:i+segment_length] for i in range(0, 16000, segment_length)]

####### TESTING - get one segment ############
#segment_wav = split_waveform[SEGMENT_NUM]
#start_sample = SEGMENT_NUM*SEGMENT_LENGTH
########################################

repcycles = []

for segment_num, segment_wav in enumerate(split_waveform):
  #segment_wav = split_waveform[SEGMENT_NUM]
  start_sample = segment_num*SEGMENT_LENGTH
  # Get the Zero-Crossings, ignoring noise
  zero_crossings = find_zerocrossings(segment_wav, start_sample, silence_threshold)
  if not len(zero_crossings) > 1:
    repcycles.append([])
    continue
  # Get the fundamental frequency using the FFT
  fft_mag, max_index = fft_max(segment_wav)

  # exclude bins outside the F0_FREQ_RANGE
  # get frequency to bin index

  top_bins = klargest_with_indices(fft_mag, 4, (LOW_F0_INDEX,HIGH_F0_INDEX))
  assert len(top_bins) > 0, "Error: failed to find a fundamental frequency"
  #top_in_range = list(filter(lambda x, h=LOW_F0, l = HIGH_F0: l <= x[0] <= h, top_bins))[0]

  # Fundamental Frequency: Sample Rate / fftsize * fft_bin_index
  f0 = AUDIO_SIZE/FFT_SIZE * top_bins[0][0]

  #print(f"silence_threshold: {silence_threshold}")
  #print(f"f0: {f0}")
  #print(f"F0 magnitude{fft_mag[max_index]}")

  #find cycles within the signal and find a representative one for the segment
  cycles = find_cycles_f0(f0, start_sample, zero_crossings)
  if not len(cycles) > 0:
    repcycles.append([])
    continue

  repcycle = find_repcycle(segment_wav, start_sample, f0, cycles)
  assert repcycle != None, "Error: failed to find a representative cycle"

  repcycles.append(repcycle)
  #show_waveform(segment_wav, start_sample, zero_crossings, repcycle)
  #show_fft(fft_mag, start_sample, 4)

show_repcycles(repcycles, waveform_arr)

#for num in cycles:
#  print(f"({num[0]:.2f}, {num[1]:.2f})")
#with open('fft_freq.txt', 'w') as file:
#  for index, value in enumerate(fft_mag):
#    file.write(f"{value} at {index*AUDIO_SIZE/FFT_SIZE}")

#### TEST- output audio 10 copies of repcycle ####
outfile_wav = []
for curr_repcycle in repcycles:
  if curr_repcycle:
    repc_start = math.floor(curr_repcycle[0])
    repc_end = math.floor(curr_repcycle[1])
    rep_cycle_wav = waveform_arr[repc_start:repc_end]
    for _ in range(10):
      outfile_wav.extend(rep_cycle_wav)
    
outfile_tensor = torch.tensor(outfile_wav).unsqueeze(0)
torchaudio.save(f"{FILE_NAME.split(".")[0]}_out.wav", outfile_tensor, 16000)
################################################################

